Ask GPT questions for one youtube video -- code doesn't perform well for large database. Working on building this out.

### Imports

In [4]:
from dotenv import load_dotenv # add this line
import os
load_dotenv()

True

In [5]:
# Import libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

from sentence_transformers import SentenceTransformer, util
import torch

/Users/kitanademidun/anaconda3/envs/wordtoobe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Convert the transcripts into text files

In [6]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('{}.json'.format(os.getenv('PROJECT_ID')))

app = firebase_admin.initialize_app(cred)

db = firestore.client(app)
batch = db.batch()

#### One-time: get the transcripts

In [7]:
channelId = "UCamLstJyCa-t5gfZegxsFMw"

In [8]:
# load the transcript text and source from data base
docs_generator = db.collection(u"WhisperTranscriptions").where(u"channelId",u"==",u"{}".format(channelId)).stream()
docs = [doc for doc in docs_generator]

In [9]:
texts = [ doc.to_dict()['text'] for doc in docs if len(doc.to_dict()['text']) > 0 ]
metadatas = [ {'source':doc.to_dict()['videoId']+'.txt'}  for doc in docs if len(doc.to_dict()['text']) > 0  ]

In [10]:
for text,metadata in zip(texts,metadatas):    
    f = open("files/"+metadata['source'],"w+")
    f.write(text)
    f.close()

Ask question to a video

In [17]:
video_id = "fyt-E1A-jwc"

In [18]:
loader = DirectoryLoader('files', glob="./"+video_id+".txt")
files = loader.load ( )
len(files)

1

In [19]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=15000, chunk_overlap=200)
documents = text_splitter.create_documents(texts, metadatas=metadatas)
split_documents = text_splitter.split_documents(files)

In [20]:
len(split_documents)

2

In [21]:
embedding = OpenAIEmbeddings()

In [22]:
vectordb= None
persist_directory = 'one_file'
vectordb = Chroma.from_documents(documents=split_documents,
                                 embedding=embedding,
                                 persist_directory=persist_directory
                                )


print("complete")
# vectordb.persist()

complete


In [23]:
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

In [24]:
chat_model = ChatOpenAI(temperature=0, model_name='gpt-4')

qa_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True
)


def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [25]:
query = "Based on the text, name all the creator brands / products launched?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
The creator brands/products mentioned in the text are:

1. David Dobryx's new frozen pizza line
2. ChamberlainCoffee.com
3. Silly Juice from the Ace Family
4. Ani Energy from the Sway Boys
5. JoJo C.1 Ice Cream
6. Father Foods from Kelsey Calamine


Sources:
files/fyt-E1A-jwc.txt
